In [ ]:
import os
import re
import numpy as np
import cv2
from collections import defaultdict
from random import randrange
from ultralytics import YOLO
from ultralytics.utils.torch_utils import select_device
from SFSORT import SFSORT

In [3]:

# Utility functions
def create_output_directory(output_path):
    try:
        os.makedirs(output_path, exist_ok=True)
        print(f"Directory created at: {output_path}")
    except OSError as e:
        print(f"Error creating directory: {e}")

def list_files(directory):
    try:
        return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    except Exception as e:
        print(f"Error listing files in directory {directory}: {e}")
        return []

def get_sorted_file_paths(directory):
    files = list_files(directory)
    return [os.path.join(directory, f"frame_{i:05d}.png") for i in range(1, len(files) + 1)]

def write_mot_format(data, filename):
    try:
        with open(filename, "w") as f:
            for entry in data:
                line = "{},{},{},{},{},{},{},{},{},{}\n".format(*entry)
                f.write(line)
        print(f"MOT data written to {filename}")
    except Exception as e:
        print(f"Error writing to file {filename}: {e}")

# Core functions
def initialize_model(model_path):
    model = YOLO(model_path, 'detect')
    device = select_device('cpu')
    model.to(device)
    return model

def process_frame(frame, model, tracker, track_history, frame_id, colors):
    prediction = model.predict(frame)
    prediction_results = prediction[0].boxes.cpu().numpy()
    tracks = tracker.update(prediction_results.xyxy, prediction_results.conf)

    if len(tracks) == 0:
        return

    bbox_list = tracks[:, 0]
    track_id_list = tracks[:, 1]

    for track_id, bbox in zip(track_id_list, bbox_list):
        if track_id not in colors:
            colors[track_id] = (randrange(255), randrange(255), randrange(255))

        x0, y0, x1, y1 = map(int, bbox)
        track_history.append((frame_id, track_id, x0, y0, x1 - x0, y1 - y0, -1, -1, -1, -1))

def process_video(dir_images, model, tracker, output_file):
    files_sorted = get_sorted_file_paths(dir_images)
    track_history = []
    colors = {}

    for file in files_sorted:
        match = re.search(r'frame_\d+\.png', file)
        if not match:
            continue

        frame_id = int(match.group(0).split("_")[1].split(".")[0])
        frame = cv2.imread(file)
        process_frame(frame, model, tracker, track_history, frame_id, colors)

    write_mot_format(track_history, output_file)

In [ ]:
# Global constants
ALGORITHM = "sfsort"
MODEL_NAME = "yolo11n_DJI_0008_V_and_0010_V_2@fine-tuning" #"yolo11x_DJI_0008_V_and_0010_V_2@fine-tuning" # yolo11n_DJI_0010_V_2@fine-tuning yolo11n_DJI_0008_V@fine-tuning
OUTPUT_PATH_MOT_FILES = f'../../output/mot_detections/{ALGORITHM}'
FRAME_RATE = 25
FRAME_WIDTH = 1920
FRAME_HEIGHT = 1088

TRACKER_ARGUMENTS = {
    "dynamic_tuning": True,
    "cth": 0.7,
    "high_th": 0.82,
    "high_th_m": 0.1,
    "match_th_first": 0.5,
    "match_th_first_m": 0.05,
    "match_th_second": 0.1,
    "low_th": 0.3,
    "new_track_th": 0.7,
    "new_track_th_m": 0.1,
    "marginal_timeout": (7 * FRAME_RATE // 10),
    "central_timeout": FRAME_RATE,
    "horizontal_margin": FRAME_WIDTH // 10,
    "vertical_margin": FRAME_HEIGHT // 10,
    "frame_width": FRAME_WIDTH,
    "frame_height": FRAME_HEIGHT
}

create_output_directory(OUTPUT_PATH_MOT_FILES)

In [ ]:
name_video = "DJI_20240308111117_0010_V_1"
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = SFSORT(TRACKER_ARGUMENTS)

process_video(dir_images, model, tracker, output_file)

In [ ]:
name_video = "DJI_20240308110228_0006_V_2"
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = SFSORT(TRACKER_ARGUMENTS)

process_video(dir_images, model, tracker, output_file)

In [ ]:
name_video = "DJI_20240308110115_0005_V"
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = SFSORT(TRACKER_ARGUMENTS)

process_video(dir_images, model, tracker, output_file)

In [ ]:
name_video = "DJI_20240308110013_0004_V_1"
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = SFSORT(TRACKER_ARGUMENTS)

process_video(dir_images, model, tracker, output_file)

In [9]:
name_video = "DJI_20240308110013_0004_V_3"
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = SFSORT(TRACKER_ARGUMENTS)

process_video(dir_images, model, tracker, output_file)

0: 384x640 10 pineapples, 230.8ms
Speed: 5.0ms preprocess, 230.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 pineapples, 190.2ms
Speed: 4.9ms preprocess, 190.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pineapples, 175.3ms
Speed: 5.4ms preprocess, 175.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 pineapples, 169.6ms
Speed: 5.9ms preprocess, 169.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 pineapples, 170.0ms
Speed: 0.0ms preprocess, 170.0ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 pineapples, 210.1ms
Speed: 2.8ms preprocess, 210.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 pineapples, 195.9ms
Speed: 2.8ms preprocess, 195.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 pineapples, 165.8ms
Speed: 0.0ms preprocess, 165.8ms in